<a href="https://colab.research.google.com/github/PhuocOng/Spam-Classifier-Email/blob/main/Text_Classification_A_Spam_Classifier_for_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00


In [3]:
!pip install transformers

In [4]:
from datasets import load_dataset
dataset = load_dataset("sms_spam")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch

In [7]:
def preprocess(text):
  inputs = tokenizer(text, padding = True, truncation = True, return_tensors = "pt")
  return inputs

In [8]:
def predict_spam(text):
  inputs = preprocess(text)
  with torch.no_grad():
    outputs = model(**inputs)

  logits = outputs.logits
  predicted_class = torch.argmax(logits, dim = 1).item()
  return predicted_class

def predict_spam_confidence(text):
  inputs = preprocess(text)
  with torch.no_grad():
    outputs = model(**inputs)
  logits = outputs.logits
  probabilities = torch.softmax(logits, dim = 1)
  predicted_class = torch.argmax(logits, dim = 1).item()
  confidence = probabilities[0, predicted_class].item()
  return confidence

In [27]:
def classify_message(text):
    class_map = {0: "Not Spam", 1: "Spam"}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Detect device
    model.to(device)  # Move model to the correct device

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to same device

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_class = torch.argmax(logits, dim=1).item()

    confidence = probabilities[0][predicted_class].item()  # Get confidence score

    return class_map[predicted_class], confidence

In [34]:
example_messages = [
    "Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.",
    "Hey, are we still meeting for coffee later?",
    "URGENT! Your account has been compromised. Reset your password immediately.",
    "Can you send me the project report by tomorrow?",
    "eiufnqbuivbeuiv",
    "Hi Peter! We have a job give you $500000 in 1 hour!!",
    "Hey, do you want to win the fantasy football league this year? Let's strategize!",
    "Hey Mom, my college is offering free lunch this Friday. Want to join?"
    "You have WON a brand new car!!! Click this link NOW to claim your prize."
    ]

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

for msg in example_messages:
    label, confidence = classify_message(msg)
    print(f"📩 Message: {msg}")
    print(f"🔍 Prediction: {label} (Confidence: {confidence:.4f})\n")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📩 Message: Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.
🔍 Prediction: Spam (Confidence: 0.5329)

📩 Message: Hey, are we still meeting for coffee later?
🔍 Prediction: Spam (Confidence: 0.5398)

📩 Message: URGENT! Your account has been compromised. Reset your password immediately.
🔍 Prediction: Spam (Confidence: 0.5136)

📩 Message: Can you send me the project report by tomorrow?
🔍 Prediction: Spam (Confidence: 0.5114)

📩 Message: eiufnqbuivbeuiv
🔍 Prediction: Spam (Confidence: 0.5348)

📩 Message: Hi Peter! We have a job give you $500000 in 1 hour!!
🔍 Prediction: Not Spam (Confidence: 0.5031)

📩 Message: Hey, do you want to win the fantasy football league this year? Let's strategize!
🔍 Prediction: Spam (Confidence: 0.5336)

📩 Message: Hey Mom, my college is offering free lunch this Friday. Want to join?You have WON a brand new car!!! Click this link NOW to claim your prize.
🔍 Prediction: Spam (Confidence: 0.5194)



In [11]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [13]:
dataset = load_dataset("sms_spam")
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})


In [60]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples["sms"], padding = "max_length", truncation=True)

tokenized_datasets = dataset.map(preprocess_function, batched = True)
print(tokenized_datasets)

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1115
    })
})


In [74]:
print(tokenized_datasets)
tokenized_datasets.set_format("torch", columns=["sms", "input_ids", "attention_mask", "labels"])
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['sms', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1115
    })
})
DatasetDict({
    train: Dataset({
        features: ['sms', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1115
    })
})


In [16]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = np.argmax(pred.predictions, axis = 1)
  acc = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average = "binary")
  return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [77]:
training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 5,
    weight_decay = 0.01,
    logging_dir = "./logs",
    logging_steps = 10,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [78]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: onggiaphuoc2004 (onggiaphuoc2004-meta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.003200,0.071586,0.986547,0.978571,0.919463,0.948097
2,0.170700,0.099740,0.985650,0.946309,0.946309,0.946309


In [20]:
trainer.save_model("./bert_spam_classifier")
tokenizer.save_pretrained("./bert_spam_classifier")

('./bert_spam_classifier/tokenizer_config.json',
 './bert_spam_classifier/special_tokens_map.json',
 './bert_spam_classifier/vocab.txt',
 './bert_spam_classifier/added_tokens.json',
 './bert_spam_classifier/tokenizer.json')

In [44]:
model = AutoModelForSequenceClassification.from_pretrained("bert_spam_classifier")
tokenizer = AutoTokenizer.from_pretrained("bert_spam_classifier")
def classify_finetuned_message(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Detect device
    model.to(device)  # Move model to the correct device

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to same device

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(logits, dim=1).item()
    confidence = probabilities[0, predicted_class].item()

    return "Spam" if predicted_class == 1 else "Not Spam", confidence



In [48]:

# Test on example messages
for msg in example_messages:
    print(f"📩 Message: {msg}")
    label, confidence = classify_finetuned_message(msg)
    print(f"📩 Message: {msg}")
    print(f"🔍 Prediction: {label} (Confidence: {confidence:.4f})\n")

📩 Message: Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.
📩 Message: Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.
🔍 Prediction: Spam (Confidence: 0.6704)

📩 Message: Hey, are we still meeting for coffee later?
📩 Message: Hey, are we still meeting for coffee later?
🔍 Prediction: Spam (Confidence: 0.7013)

📩 Message: URGENT! Your account has been compromised. Reset your password immediately.
📩 Message: URGENT! Your account has been compromised. Reset your password immediately.
🔍 Prediction: Spam (Confidence: 0.6562)

📩 Message: Can you send me the project report by tomorrow?
📩 Message: Can you send me the project report by tomorrow?
🔍 Prediction: Spam (Confidence: 0.6912)

📩 Message: eiufnqbuivbeuiv
📩 Message: eiufnqbuivbeuiv
🔍 Prediction: Spam (Confidence: 0.7264)

📩 Message: Hi Peter! We have a job give you $500000 in 1 hour!!
📩 Message: Hi Peter! We have a job give you $500000 in 1 hour!!
🔍 Prediction: Spam (Confidenc

In [65]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def evaluate_model(model, tokenizer, test_dataset):
  all_preds = []
  all_labels = []
  for example in test_dataset:
    text = example["sms"]
    true_label = example["labels"]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
      outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    all_preds.append(predicted_class)
    all_labels.append(true_label)

  accuracy = accuracy_score(all_labels, all_preds)
  precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1_score": f1
  }

In [76]:
original_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
original_model.to("cuda" if torch.cuda.is_available() else "cpu")

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("bert_spam_classifier")
fine_tuned_model.to("cuda" if torch.cuda.is_available() else "cpu")

print(tokenized_datasets["test"])
original_results = evaluate_model(original_model, tokenizer, tokenized_datasets["test"])
fine_tuned_results = evaluate_model(fine_tuned_model, tokenizer, tokenized_datasets["test"])

# Print results
print("📊 **Original Model (Untrained BERT) Performance:**")
print(original_results)

print("\n🚀 **Fine-Tuned Model Performance:**")
print(fine_tuned_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset({
    features: ['sms', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1115
})
📊 **Original Model (Untrained BERT) Performance:**
{'accuracy': 0.8645739910313901, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}

🚀 **Fine-Tuned Model Performance:**
{'accuracy': 0.1336322869955157, 'precision': 0.1336322869955157, 'recall': 1.0, 'f1_score': 0.23575949367088608}


In [ ]:
import collections
label_counts = collections.Counter(tokenized_datasets["train"]["labels"])
print(label_counts)